<a href="https://colab.research.google.com/github/kiran1442/ColorFusionMetrics/blob/main/Assignment_4_imageColorization_nullClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torchvision
import cv2
import numpy as np
from PIL import Image

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    # transforms.Resize((256, 256)),
    transforms.ToTensor()
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset,shuffle=True,batch_size=64,num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset,shuffle=False,batch_size=64,num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class UNetColorization(nn.Module):
    def __init__(self):
        super(UNetColorization, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            # Remove MaxPool2d or use Upsample in the decoder to maintain the original size
            # nn.MaxPool2d(2)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 3, 3, padding=1),
            nn.Sigmoid(),
            # Add Upsample to restore the original size if MaxPool2d is used in the encoder
            # nn.Upsample(scale_factor=2, mode='bilinear')
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
model = UNetColorization().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# def rgb_to_gray(img):
#     return img.mean(dim=1, keepdim=True)

In [ ]:
num_epochs = 1
for epoch in range(num_epochs):
    for i, (grayscale, _) in enumerate(train_loader):
        grayscale = grayscale.to(device)
        optimizer.zero_grad()
        output = model(grayscale)
        loss = criterion(output, grayscale.expand(-1, 3, -1, -1))  # Match shape dynamically
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
          print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Save trained model
torch.save(model.state_dict(), "unet_model.pth")


Epoch [1/1], Step [1/782], Loss: 0.0002
Epoch [1/1], Step [101/782], Loss: 0.0002
Epoch [1/1], Step [201/782], Loss: 0.0001
Epoch [1/1], Step [301/782], Loss: 0.0001
Epoch [1/1], Step [401/782], Loss: 0.0001
Epoch [1/1], Step [501/782], Loss: 0.0000
Epoch [1/1], Step [601/782], Loss: 0.0001
Epoch [1/1], Step [701/782], Loss: 0.0000


In [ ]:
def load_style_model(style_path):
    style_model = cv2.dnn.readNetFromTorch(style_path)
    return style_model

# Apply Style Transfer
def apply_style_transfer(image, model):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    h, w, c = image.shape
    blob = cv2.dnn.blobFromImage(image, 1.0, (w, h), (103.939, 116.779, 123.68), swapRB=False, crop=False)
    model.setInput(blob)
    output = model.forward()
    output = output.reshape((c, h, w))
    output[0] += 103.939
    output[1] += 116.779
    output[2] += 123.68
    output = output.transpose(1, 2, 0)
    return np.clip(output, 0, 255).astype('uint8')

In [ ]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st

st.title("Hello, Streamlit on Google Colab!")
st.write("This is a simple Streamlit app running inside Google Colab.")
st.title("Grayscale Image Colorization & Style Transfer")
uploaded_file = st.file_uploader("Upload a grayscale image", type=["png", "jpg", "jpeg"])

styles = {"Van Gogh": "vangogh.pth", "Monet": "monet.pth", "Picasso": "picasso.pth"}  # Replace with real model paths
selected_style = st.selectbox("Select Artistic Style", list(styles.keys()))

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    model.load_state_dict(torch.load("unet_model.pth", map_location=torch.device('cpu')))
    model.eval()

    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    img_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(img_tensor)
    colorized_array = output.squeeze(0).permute(1, 2, 0).numpy()
    colorized_image = Image.fromarray((colorized_array * 255).astype(np.uint8))

    style_model = load_style_model(styles[selected_style])
    stylized_image = apply_style_transfer(colorized_image, style_model)

    st.image(stylized_image, caption="Colorized & Styled Image", use_column_width=True)

    result = Image.fromarray(stylized_image)
    st.download_button("Download Image", data=result.tobytes(), file_name="styled_image.jpg", mime="image/jpeg")



Overwriting app.py


In [ ]:
from pyngrok import ngrok

# Start Streamlit in the background
!streamlit run app.py &

# Expose the Streamlit app to the internet
public_url = ngrok.connect(port="8501")
print(f"Streamlit app running at: {public_url}")




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.202.179.150:8501

  Stopping...


ERROR:pyngrok.process.ngrok:t=2025-03-11T06:36:07+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-03-11T06:36:07+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-03-11T06:36:07+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.